In [1]:
import nltk
import re
import pymorphy2
from collections import Counter
nltk.download('stopwords')
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier, RidgeClassifierCV, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from IPython.core.display import HTML, display
from unidecode import unidecode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\User\AppData\Local\Temp\ipykernel_6012\2102828800.py:23: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [4]:
len(test_df)

165378

In [5]:
train_df["target"].value_counts()

target
False    118594
True      16715
Name: count, dtype: int64

In [6]:
from sklearn.metrics import f1_score

In [7]:
X_train = train_df[["url", "title"]]
X_test = test_df[["url", "title"]]
y_train = train_df["target"].astype(int).values

# Препроцессинг данных

## Токенизация

In [8]:
TOKEN_PATTERN = "[a-zа-яё]+"

def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())

docs_url = [tokenize((str(text))) for text in X_train["url"]]
docs_title = [tokenize(str(text)) for text in X_train["title"]]

In [9]:
docs_url

[['m', 'kp', 'md'],
 ['www', 'kp', 'by'],
 ['fanserials', 'tv'],
 ['colorbox', 'spb', 'ru'],
 ['tula', 'sport', 'ru'],
 ['beregifiguru', 'ru'],
 ['ekb', 'vseinstrumenti', 'ru'],
 ['mirtabaka', 'org'],
 ['xlecx', 'com'],
 ['bus', 'biletyplus', 'ua'],
 ['otvet', 'gotov', 'ru'],
 ['mail', 'ru'],
 ['pornmult', 'info'],
 ['libking', 'ru'],
 ['www', 'idealmedia', 'io'],
 ['mail', 'ru'],
 ['zaycev', 'net'],
 ['kazan', 'ucheba', 'ru'],
 ['mail', 'ru'],
 ['eropixel', 'net'],
 ['royallib', 'com'],
 ['gdespaces', 'com'],
 ['ulyanovsk', 'beeline', 'ru'],
 ['xorazm', 'net'],
 ['mail', 'ru'],
 ['www', 'gismeteo', 'ua'],
 ['corel', 'ru'],
 ['www', 'mvideo', 'ru'],
 ['apteka', 'ru'],
 ['norma', 'org', 'ua'],
 ['mychords', 'net'],
 ['deal', 'by'],
 ['www', 'farpost', 'ru'],
 ['www', 'mvideo', 'ru'],
 ['m', 'auto', 'n', 'ru'],
 ['vtk', 'moscow', 'ru'],
 ['tengrinews', 'kz'],
 ['mail', 'ru'],
 ['www', 'lamnia', 'com'],
 ['mmag', 'ru'],
 ['otzovik', 'com'],
 ['hdxclub', 'com'],
 ['infourok', 'ru'],
 ['tas

In [10]:
docs_title

[['экс',
  'министр',
  'экономики',
  'молдовы',
  'главе',
  'мидэи',
  'цель',
  'которого',
  'сделать',
  'из',
  'республики',
  'не',
  'просителя',
  'а',
  'донора',
  'надо',
  'избегать',
  'долгого',
  'нахождения',
  'н'],
 ['эта',
  'песня',
  'стала',
  'известна',
  'многим',
  'телезрителям',
  'благодаря',
  'сериалу',
  'диверсант'],
 ['банши', 'сезон', 'серия', 'бремя', 'красоты', 'смотреть', 'онлайн'],
 ['не', 'беси', 'меня', 'картинки'],
 ['в',
  'новомосковске',
  'сыграют',
  'следж',
  'хоккеисты',
  'алексинской',
  'звезды',
  'и',
  'сборной',
  'китая',
  'т'],
 ['салат',
  'корейская',
  'морковь',
  'копченая',
  'курица',
  'кукуруза',
  'сухарики',
  'мой',
  'калорийность',
  'белки',
  'жиры',
  'углеводы'],
 ['угловой',
  'пневмогайковерт',
  'fubag',
  'rwc',
  'в',
  'екатеринбурге',
  'купить',
  'цены',
  'отзывы',
  'характеристики',
  'фото',
  'инструкция'],
 ['табачный',
  'магазин',
  'мир',
  'табака',
  'курительные',
  'трубки',
  'stanwe

In [11]:
occurence = Counter([token for doc in docs_title for token in doc])
occurence

Counter({'экс': 44,
         'министр': 11,
         'экономики': 38,
         'молдовы': 15,
         'главе': 8,
         'мидэи': 1,
         'цель': 39,
         'которого': 23,
         'сделать': 235,
         'из': 3711,
         'республики': 111,
         'не': 2282,
         'просителя': 1,
         'а': 1692,
         'донора': 1,
         'надо': 80,
         'избегать': 4,
         'долгого': 2,
         'нахождения': 4,
         'н': 998,
         'эта': 33,
         'песня': 111,
         'стала': 51,
         'известна': 6,
         'многим': 1,
         'телезрителям': 1,
         'благодаря': 10,
         'сериалу': 5,
         'диверсант': 2,
         'банши': 3,
         'сезон': 1014,
         'серия': 1096,
         'бремя': 5,
         'красоты': 169,
         'смотреть': 5542,
         'онлайн': 11969,
         'беси': 1,
         'меня': 216,
         'картинки': 549,
         'в': 38391,
         'новомосковске': 3,
         'сыграют': 3,
         'следж': 1,


### Загрузка стоп-слов

In [12]:
from nltk.corpus import stopwords as stopwrds
stopwrds.fileids()

['arabic',
 'azerbaijani',
 'basque',
 'bengali',
 'catalan',
 'chinese',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hebrew',
 'hinglish',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [13]:
nltk.corpus.stopwords.words('turkish')

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [14]:
stopwords = []
for language in ['russian', 'english', 'portuguese', 'spanish', 'tajik', 'turkish']:
    stopwords += nltk.corpus.stopwords.words(language)
stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

### Лемматизация

In [15]:
lemmatizer = pymorphy2.MorphAnalyzer()

In [16]:
lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm_notebook(docs_title)]

cleared_docs = [[token for token in text if token not in stopwords] for text in tqdm_notebook(lemmatized_docs)]

C:\Users\User\AppData\Local\Temp\ipykernel_6012\2321608028.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm_notebook(docs_title)]


  0%|          | 0/135309 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp\ipykernel_6012\2321608028.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  cleared_docs = [[token for token in text if token not in stopwords] for text in tqdm_notebook(lemmatized_docs)]


  0%|          | 0/135309 [00:00<?, ?it/s]

In [17]:
docs_title

[['экс',
  'министр',
  'экономики',
  'молдовы',
  'главе',
  'мидэи',
  'цель',
  'которого',
  'сделать',
  'из',
  'республики',
  'не',
  'просителя',
  'а',
  'донора',
  'надо',
  'избегать',
  'долгого',
  'нахождения',
  'н'],
 ['эта',
  'песня',
  'стала',
  'известна',
  'многим',
  'телезрителям',
  'благодаря',
  'сериалу',
  'диверсант'],
 ['банши', 'сезон', 'серия', 'бремя', 'красоты', 'смотреть', 'онлайн'],
 ['не', 'беси', 'меня', 'картинки'],
 ['в',
  'новомосковске',
  'сыграют',
  'следж',
  'хоккеисты',
  'алексинской',
  'звезды',
  'и',
  'сборной',
  'китая',
  'т'],
 ['салат',
  'корейская',
  'морковь',
  'копченая',
  'курица',
  'кукуруза',
  'сухарики',
  'мой',
  'калорийность',
  'белки',
  'жиры',
  'углеводы'],
 ['угловой',
  'пневмогайковерт',
  'fubag',
  'rwc',
  'в',
  'екатеринбурге',
  'купить',
  'цены',
  'отзывы',
  'характеристики',
  'фото',
  'инструкция'],
 ['табачный',
  'магазин',
  'мир',
  'табака',
  'курительные',
  'трубки',
  'stanwe

In [18]:
cooccurence_title = Counter([(doc[i], doc[i + 1]) for doc in docs_title for i in range(len(doc) - 1)])

In [19]:
cooccurence_title.most_common(20)

[(('mail', 'ru'), 6701),
 (('смотреть', 'онлайн'), 3820),
 (('поиск', 'mail'), 3502),
 (('результатов', 'поиск'), 2622),
 (('купить', 'в'), 2107),
 (('скачать', 'бесплатно'), 2012),
 (('онлайн', 'бесплатно'), 1957),
 (('читать', 'онлайн'), 1808),
 (('в', 'хорошем'), 1629),
 (('тыс', 'результатов'), 1611),
 (('порно', 'видео'), 1529),
 (('в', 'интернет'), 1529),
 (('в', 'москве'), 1489),
 (('интернет', 'магазине'), 1419),
 (('интернет', 'магазин'), 1377),
 (('хорошем', 'качестве'), 1361),
 (('бесплатно', 'в'), 1315),
 (('на', 'olx'), 1289),
 (('мой', 'мир'), 1123),
 (('мир', 'mail'), 1054)]

In [20]:
cooccurence_url = Counter([(doc[i], doc[i + 1]) for doc in docs_url for i in range(len(doc) - 1)])

In [21]:
cooccurence_url.most_common(20)

[(('mail', 'ru'), 7906),
 (('drom', 'ru'), 2272),
 (('xn', 'p'), 1455),
 (('p', 'ai'), 1447),
 (('com', 'ua'), 1258),
 (('www', 'olx'), 1234),
 (('baza', 'drom'), 1042),
 (('olx', 'ua'), 949),
 (('irecommend', 'ru'), 767),
 (('cdn', 'ampproject'), 751),
 (('ampproject', 'org'), 751),
 (('youla', 'ru'), 750),
 (('hh', 'ru'), 745),
 (('daftsex', 'com'), 741),
 (('babyblog', 'ru'), 721),
 (('cian', 'ru'), 688),
 (('fotostrana', 'ru'), 687),
 (('s', 'mobi'), 669),
 (('v', 's'), 668),
 (('biqle', 'ru'), 658)]

### Создание обработанного датасета

In [22]:
cleared_docs_sentence = [' '.join(text) for text in cleared_docs]

In [23]:
docs_url_sentence = [' '.join(text) for text in docs_url]

In [24]:
d = {'url':docs_url_sentence, 'title': cleared_docs_sentence, 'target': y_train}

In [25]:
train_dataset = pd.DataFrame(data=d)
train_dataset

,url,title,target
0,m kp md,экс министр экономика молдова глава мидэи цель...,0
1,www kp by,песня стать известный многий телезритель благо...,0
2,fanserials tv,банши сезон серия бремя красота смотреть онлайн,0
3,colorbox spb ru,бесить картинка,0
4,tula sport ru,новомосковск сыграть следж хоккеист алексински...,0
...,...,...,...
135304,mail ru,пора тюльпан турецкий сериал русский язык резу...,0
135305,www ntv ru,остросюжетный сериал шеф игра повышение серия,0
135306,topclassiccarsforsale com,plymouth special deluxe hot rod automatic smal...,0
135307,wowcream ru,купить skin сыворотка питательный power formul...,0


In [26]:
SEED = 42
df_train, df_val = train_test_split(train_dataset, test_size=0.2, random_state=SEED)

In [27]:
df_train

,url,title,target
36898,karat spb ru,амиксидин продажа цена санкт петербург медицин...,0
69667,samara farfor ru,ультрасет new,0
111124,www zr ru,lada vesta cross видеообзор презентация серийн...,0
35818,xn htbdmodofzkc c xn p ai,фрезерный станок моделист cnc х мм продажа цен...,0
123149,porno co,популярный порно просмотр порнуха популярный с...,1
...,...,...,...
110268,pornophoto pro,смотреть бритый вагин фото,1
119879,multilisting su,продать трехкомнатную вторичку мкр олимпийский...,0
103694,digital k by,sharp dg купить минск k,0
131932,www citilink ru,купить web камера logitech conferencecam rally...,0


In [28]:
df_val

,url,title,target
54589,www kinotrast com,сериал качели беларусь весь серия смотреть онл...,0
41071,pc ru,шкаф спальный гарнитур энрике рубль грозный ан...,0
19161,forum zoneofgames ru,swat close quarters battle русификатор zone ga...,0
75429,librebook me,читать онлайн электронный книга прометей жизнь...,0
121091,meduniver com,научить щедрость собственный мужчина отучить м...,0
...,...,...,...
67849,sdelairukami ru,деревянный настольный лампа трансформер свой рука,0
30903,agroolkar com ua,кормовой добавка сила природа дойный корова l ...,0
43564,lyricshare net,текст песнь софия ротару белый зима слово песнь,0
67387,worldserials org,защитник protector сезон смотреть сериал онлай...,0


In [29]:
df_train[df_train['target'] == 1].shape[0] / df_train.shape[0]

0.12356000628192929

In [30]:
df_val[df_val['target'] == 1].shape[0] / df_val.shape[0]

0.12342029413938364

## Объединение признаков

In [31]:
def preprocessor(text):
    whitespaced_text = re.sub("[^a-zа-яё]", ' ', text.lower())
    return re.sub(' +', ' ',  whitespaced_text)

In [32]:
pipeline = Pipeline([
    (
        'features', 
        ColumnTransformer([
            (
                'url', 
                TfidfVectorizer(
            lowercase=True, ngram_range=(3, 6), analyzer='char',
            preprocessor=preprocessor, min_df=3, max_df=0.5
        ), 
                'url'
            ),
            (
                'title',
                TfidfVectorizer(
            lowercase=True, ngram_range=(3, 6), analyzer='char',
            preprocessor=preprocessor, min_df=3, max_df=0.5
        ), 
                'title'
            )
        ])
    ),
    ('clf', SGDClassifier(random_state=SEED, loss='modified_huber', class_weight='balanced'))
])

In [33]:
x_train = df_train[['url', 'title']]
y_train = df_train['target'].values

In [34]:
x_val = df_val[['url', 'title']]
y_val = df_val['target'].values

In [35]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('url',
                                                  TfidfVectorizer(analyzer='char',
                                                                  max_df=0.5,
                                                                  min_df=3,
                                                                  ngram_range=(3,
                                                                               6),
                                                                  preprocessor=<function preprocessor at 0x000001FD48E77400>),
                                                  'url'),
                                                 ('title',
                                                  TfidfVectorizer(analyzer='char',
                                                                  max_df=0.5,
                                                                  min_df=3,
                                                                  ngram_range=(3,
                                                                               6),
                                                                  preprocessor=<function preprocessor at 0x000001FD48E77400>),
                                                  'title')])),
                ('clf',
                 SGDClassifier(class_weight='balanced', loss='modified_huber',
                               random_state=42))])

In [36]:
f1_score(
    y_train,
    pipeline.predict(df_train)
)

0.994973751815034

In [37]:
f1_score(
    y_val,
    pipeline.predict(df_val)
)

0.9845739104388198

## Обработака тестового датасета

In [38]:
docs_test_url = [tokenize(str(text)) for text in X_test['url']]
docs_test_title = [tokenize(str(text)) for text in X_test['title']]

In [39]:
lemmatized_docs_test = [[lemmatize(token) for token in text] for text in tqdm_notebook(docs_test_title)]

cleared_docs_test = [[token for token in text if token not in stopwords] for text in lemmatized_docs_test]

C:\Users\User\AppData\Local\Temp\ipykernel_6012\250155301.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmatized_docs_test = [[lemmatize(token) for token in text] for text in tqdm_notebook(docs_test_title)]


  0%|          | 0/165378 [00:00<?, ?it/s]

In [40]:
cleared_docs_test

[['шесть',
  'кассационный',
  'суд',
  'самар',
  'начать',
  'работа',
  'разный',
  'здание',
  'фото',
  'коммерсантъ'],
 ['индексация',
  'алименты',
  'случай',
  'производиться',
  'каков',
  'порядок',
  'правило',
  'процедура'],
 ['женщина', 'империя', 'мех', 'part'],
 ['небритый',
  'волосатый',
  'киска',
  'порно',
  'весь',
  'страна',
  'национальность',
  'онлайн'],
 [],
 ['вакансия',
  'мерчендайзер',
  'визитный',
  'рязань',
  'группа',
  'компания',
  'open',
  'поиск',
  'работа',
  'городработ',
  'ру'],
 ['смартфон',
  'apple',
  'iphone',
  'xr',
  'gb',
  'чёрный',
  'купить',
  'интернет',
  'магазин',
  'фотосклад',
  'ру',
  'цена',
  'отзыв',
  'видео',
  'обзор'],
 ['духов',
  'шкаф',
  'siemens',
  'hb',
  'gys',
  'r',
  'sim',
  'dealer',
  'цена',
  'москва'],
 ['вакансия',
  'аналитик',
  'департамент',
  'внедрение',
  'информационный',
  'система',
  'crm',
  'платформа'],
 ['anomaly', 'defenders', 'v', 'торрент'],
 ['убрать', 'запах', 'выгребной', 

In [41]:
cleared_docs_test_sentence = [' '.join(text) for text in cleared_docs_test]

In [42]:
docs_test_url_sentence = [' '.join(text) for text in docs_test_url]

In [43]:
data_test = {'url':docs_test_url_sentence, 'title': cleared_docs_test_sentence}

In [44]:
test_dataset = pd.DataFrame(data=data_test)
test_dataset

,url,title
0,www kommersant ru,шесть кассационный суд самар начать работа раз...
1,urexpert online,индексация алименты случай производиться каков...
2,imperimeha ru,женщина империя мех part
3,national porn com,небритый волосатый киска порно весь страна нац...
4,gis ru,
...,...,...
165373,etp armtek ru,armtek запчасть грузовой легковой автомобиль о...
165374,mail ru,лилия якупова караганда карагандинский область...
165375,xn sbnqchpeeeth xn p ai,администрация лесной район тверская область го...
165376,www sunhome ru cdn ampproject org,сонник изменение сознание сниться изменение со...


In [45]:
ans = pipeline.predict(test_dataset).astype(bool)

In [46]:
data = {'id': test_df['id'], 'target': ans}

In [47]:
pd_ans = pd.DataFrame(data=data)

In [48]:
pd_ans

,id,target
0,135309,False
1,135310,False
2,135311,False
3,135312,True
4,135313,False
...,...,...
165373,300682,False
165374,300683,False
165375,300684,False
165376,300685,False


In [49]:
pd_ans[["id", "target"]].to_csv("ml_baseline.csv", index=False)

In [50]:
pd_ans.shape

(165378, 2)